In [67]:
import pandas as pd

# Rutas de los archivos
archivo_entrega_4 = "/content/drive/MyDrive/Entrega 4/Corregido_Final.csv"
archivo_entrega_5 = "/content/drive/MyDrive/Entrega 5/Distintos_artistas.csv"

# Lee el archivo CSV
df = pd.read_csv(archivo_entrega_4)

# Divide la columna 'artist_names' en filas separadas
df = df.assign(artist_names=df['artist_names'].str.split(','))

# Convierte las listas en columnas separadas
df = df.explode('artist_names')

# Reinicia los índices del DataFrame
df = df.reset_index(drop=True)

# Guarda el DataFrame resultante en otro archivo CSV
df.to_csv(archivo_entrega_5, index=False)




In [68]:
import pandas as pd

# Ruta del archivo Distintos_artistas.csv
archivo_distintos_artistas = "/content/drive/MyDrive/Entrega 5/Distintos_artistas.csv"

# Lee el archivo CSV
df_distintos_artistas = pd.read_csv(archivo_distintos_artistas)

# Filtra los datos para artistas con valores en la columna 'año' de 2020 y 2022
df_2020_2022 = df_distintos_artistas[df_distintos_artistas['año'].isin([2020, 2022])]

# Filtra los datos para artistas con valores en la columna 'año' de 2021 y 2022
df_2021_2022 = df_distintos_artistas[df_distintos_artistas['año'].isin([2021, 2022])]

# Guarda los DataFrames resultantes en archivos CSV
archivo_2020_2022 = "/content/drive/MyDrive/Entrega 5/artistas_2020_2022.csv"
archivo_2021_2022 = "/content/drive/MyDrive/Entrega 5/artistas_2021_2022.csv"

df_2020_2022.to_csv(archivo_2020_2022, index=False)
df_2021_2022.to_csv(archivo_2021_2022, index=False)




Los productores o artistas extranjeros se sacaron manualmente, no funcionó el código.

In [65]:
import plotly.express as px
import pandas as pd

# Cargar los datos desde el archivo CSV
file_path = '/content/drive/MyDrive/Entrega 5/2021Artistas2022.csv'
df = pd.read_csv(file_path)

# Agrupar los datos para los dos artistas llamados "Cris Mj"
df_grouped = df.groupby(['artist_names', 'año', 'País']).size().reset_index(name='count')

# Combinar los dos artistas "Cris Mj"
df_grouped.loc[df_grouped['artist_names'] == 'Cris Mj', 'artist_names'] = 'Cris Mj (combinado)'

# Filtrar datos solo para los años 2021 y 2022
df_filtered = df_grouped[df_grouped['año'].isin([2021, 2022])]

# Ordenar el DataFrame por el número total de países en orden ascendente
order = df_filtered.groupby('artist_names')['count'].sum().sort_values().index
df_filtered['artist_names'] = pd.Categorical(df_filtered['artist_names'], categories=order, ordered=True)

# Crear la visualización con Plotly
fig = px.bar(df_filtered, x='artist_names', y='count', color='País', facet_col='año',
             facet_col_wrap=2,  # Número de columnas en la faceta
             title='Comparación de países distintos por artista en 2021 y 2022',
             labels={'count': 'Número de países', 'artist_names': 'Artista'})

# Mostrar la visualización
fig.show()

fig.write_html('/content/drive/MyDrive/Entrega 5/grafico_2021-2022.html')



In [66]:
import plotly.express as px
import pandas as pd

# Cargar los datos desde el archivo CSV
file_path = '/content/drive/MyDrive/Entrega 5/2020Artistas2022.csv'
df = pd.read_csv(file_path)

# Asegúrate de incluir siempre a "Harry Nach" y "Paloma Mami+"
artistas_incluidos = ['Cris Mj', 'Harry Nach', 'Paloma Mami']

# Agrupar los datos para los artistas
df_grouped = df.groupby(['artist_names', 'año', 'País']).size().reset_index(name='count')

# Combinar los dos artistas "Cris Mj"
df_grouped.loc[df_grouped['artist_names'] == 'Cris Mj', 'artist_names'] = 'Cris Mj (combinado)'

# Filtrar datos solo para los años 2020 y 2022
df_filtered = df_grouped[df_grouped['año'].isin([2020, 2022])]

# Crear una lista de DataFrames que incluyan los artistas siempre
dfs = []
for artista in artistas_incluidos:
    df_artista = df_filtered[df_filtered['artist_names'] == artista].copy()
    dfs.append(df_artista)

# Concatenar los DataFrames
df_combined = pd.concat([df_filtered] + dfs)

# Ordenar el DataFrame por el número total de países en orden ascendente
order = df_combined.groupby('artist_names')['count'].sum().sort_values().index
df_combined['artist_names'] = pd.Categorical(df_combined['artist_names'], categories=order, ordered=True)

# Crear la visualización con Plotly
fig = px.bar(df_combined, x='artist_names', y='count', color='País', facet_col='año',
             facet_col_wrap=2,  # Número de columnas en la faceta
             title='Comparación de países distintos por artista en 2020 y 2022',
             labels={'count': 'Número de países', 'artist_names': 'Artista', 'año': 'Año'})

# Mostrar la visualización
fig.show()

fig.write_html('/content/drive/MyDrive/Entrega 5/grafico_2020-2022.html')



In [ ]:
import pandas as pd
import plotly.express as px

# Ruta del nuevo archivo CSV
file_path = '/content/drive/MyDrive/Entrega 4/sin_duplicados.csv'

# Leer el DataFrame desde el nuevo archivo CSV
df = pd.read_csv(file_path)

# Supongamos que tienes una columna 'País' en tu DataFrame
# Cambia 'País' por el nombre real de la columna si es diferente
df_filtered = df[df['País'] != 'Chile']

# Agrupa por año y país, cuenta la cantidad de canciones en cada grupo
grouped_data = df_filtered.groupby(['año', 'País']).size().unstack(fill_value=0).reset_index()

# Crea el gráfico de barras apiladas con Plotly Express
fig = px.bar(grouped_data, x='año', y=grouped_data.columns[1:], barmode='stack',
             title='Distribución de canciones por año y país (excluyendo Chile)',
             labels={'value': 'Número de canciones', 'año': 'Año', 'variable': 'País'},
             category_orders={'año': sorted(grouped_data['año'].unique())},
             color_discrete_sequence=px.colors.sequential.Viridis)

# Ajustes de diseño
fig.update_layout(xaxis=dict(tickmode='linear', dtick=1, tickangle=45),
                  yaxis=dict(tickmode='linear', dtick=1),
                  legend_title='País',
                  legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

# Mostrar la figura interactiva
fig.show()

In [69]:
import pandas as pd
import plotly.express as px

# Ruta del nuevo archivo CSV
file_path = '/content/drive/MyDrive/Entrega 4/sin_duplicados.csv'

# Leer el DataFrame desde el nuevo archivo CSV
df = pd.read_csv(file_path)

# Supongamos que tienes una columna 'País' en tu DataFrame
# Cambia 'País' por el nombre real de la columna si es diferente

# Agrupa por año y país, cuenta la cantidad de canciones en cada grupo
grouped_data = df.groupby(['año', 'País']).size().unstack(fill_value=0).reset_index()

# Crea el gráfico de barras apiladas con Plotly Express
fig = px.bar(grouped_data, x='año', y=grouped_data.columns[1:], barmode='stack',
             title='Distribución de canciones por año y país',
             labels={'value': 'Número de canciones', 'año': 'Año', 'variable': 'País'},
             category_orders={'año': sorted(grouped_data['año'].unique())},
             color_discrete_sequence=px.colors.sequential.Viridis)

# Ajustes de diseño
fig.update_layout(xaxis=dict(tickmode='linear', dtick=1, tickangle=45),
                  yaxis=dict(tickmode='linear', dtick=10),  # Ajuste para subir de 10 en 10
                  legend_title='País',
                  legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))

# Exportar la figura como un archivo HTML
fig.write_html('/content/drive/MyDrive/Entrega 5/grafico_Distribución.html')
